<a href="https://colab.research.google.com/github/dung24mse13190/mle-final-new/blob/main/DungTNT_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras

## Load data into dataframes

In [2]:
import pandas as pd


df_train = pd.read_csv("https://raw.githubusercontent.com/dung24mse13190/mle-final-new/main/medical_train_dataset.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/dung24mse13190/mle-final-new/main/medical_test_dataset.csv")

## Summarize DataFrames

In [3]:
from sklearn.ensemble import IsolationForest

print("Summary of the training dataset:")
display(df_train.describe())
print("\nInformation about the training dataset:")
display(df_train.info())

print("\nSummary of the test dataset:")
display(df_test.describe())
print("\nInformation about the test dataset:")
display(df_test.info())

# ============================================
# 2. ADD OUTLIER DETECTION HERE (NEW CODE)
# ============================================
print("\n=== OUTLIER DETECTION ===")

# Quick outlier detection function (simplified version)
def detect_and_handle_outliers(df_train, df_test, strategy='cap'):
    """
    Simplified outlier detection and handling
    """
    continuous_cols = ['age', 'bmi', 'systolic_bp', 'diastolic_bp',
                      'heart_rate', 'blood_glucose', 'cholesterol',
                      'medication_count', 'symptoms_score']

    # 1. Use Isolation Forest for detection
    iso_forest = IsolationForest(contamination=0.05, random_state=42)

    # Fit on continuous features only
    outlier_predictions = iso_forest.fit_predict(df_train[continuous_cols].fillna(df_train[continuous_cols].median()))
    outliers = outlier_predictions == -1

    print(f"Outliers detected: {outliers.sum()} ({outliers.sum()/len(df_train)*100:.2f}%)")

    # 2. Check outliers by class
    print("\nOutliers by diagnosis class:")
    for diagnosis in sorted(df_train['diagnosis_code'].unique()):
        class_mask = df_train['diagnosis_code'] == diagnosis
        class_outliers = outliers & class_mask
        print(f"  Class {diagnosis}: {class_outliers.sum()} outliers")

    # 3. Handle outliers using capping strategy
    df_train_cleaned = df_train.copy()
    df_test_cleaned = df_test.copy()

    if strategy == 'cap':
        print("\nCapping outliers to 1st/99th percentile...")
        for col in continuous_cols:
            # Calculate bounds from training data
            lower_bound = df_train[col].quantile(0.01)
            upper_bound = df_train[col].quantile(0.99)

            # Apply to both train and test
            df_train_cleaned[col] = df_train[col].clip(lower_bound, upper_bound)
            df_test_cleaned[col] = df_test[col].clip(lower_bound, upper_bound)

            # Report capped values
            train_capped = ((df_train[col] < lower_bound) | (df_train[col] > upper_bound)).sum()
            if train_capped > 0:
                print(f"  {col}: capped {train_capped} values")

    return df_train_cleaned, df_test_cleaned

# Apply outlier detection and handling
df_train, df_test = detect_and_handle_outliers(df_train, df_test, strategy='cap')


Summary of the training dataset:


,age,bmi,systolic_bp,diastolic_bp,heart_rate,blood_glucose,cholesterol,medication_count,symptoms_score,diagnosis_code
count,10000.00000,9900.000000,10000.000000,10000.000000,10000.00000,9900.000000,9900.000000,9900.000000,9900.000000,10000.000000
mean,57.06940,27.263798,132.033800,98.620100,89.39450,151.077879,206.561818,2.505758,5.478141,0.485300
std,12.37299,4.161762,17.403883,12.606144,13.00053,42.782881,32.406536,1.722215,2.172997,0.664852
min,18.00000,16.000000,90.000000,60.000000,55.00000,70.000000,120.000000,0.000000,0.000000,0.000000
25%,49.00000,24.500000,120.000000,90.000000,81.00000,115.000000,185.000000,1.000000,4.000000,0.000000
50%,57.00000,27.200000,132.000000,99.000000,89.00000,158.000000,206.000000,2.000000,5.500000,0.000000
75%,65.00000,30.100000,144.000000,108.000000,98.00000,185.000000,228.000000,3.000000,7.000000,1.000000
max,90.00000,42.900000,194.000000,120.000000,120.00000,274.000000,328.000000,12.000000,10.000000,2.000000



Information about the training dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               10000 non-null  int64  
 1   gender            10000 non-null  object 
 2   bmi               9900 non-null   float64
 3   systolic_bp       10000 non-null  int64  
 4   diastolic_bp      10000 non-null  int64  
 5   heart_rate        10000 non-null  int64  
 6   blood_glucose     9900 non-null   float64
 7   cholesterol       9900 non-null   float64
 8   smoking_status    10000 non-null  object 
 9   exercise_level    10000 non-null  object 
 10  family_history    10000 non-null  object 
 11  medication_count  9900 non-null   float64
 12  symptoms_score    9900 non-null   float64
 13  diagnosis         10000 non-null  object 
 14  diagnosis_code    10000 non-null  int64  
dtypes: float64(5), int64(5), object(5)
memory usage

None


Summary of the test dataset:


,age,bmi,systolic_bp,diastolic_bp,heart_rate,blood_glucose,cholesterol,medication_count,symptoms_score,diagnosis_code
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,59.616667,27.099900,136.134333,101.211667,90.217667,132.042667,216.776333,2.958667,5.654533,1.000000
std,12.378023,4.041658,17.219789,12.103926,13.136228,41.173729,34.561025,1.924455,2.298518,0.816633
min,18.000000,16.000000,90.000000,60.000000,55.000000,70.000000,120.000000,0.000000,0.000000,0.000000
25%,51.000000,24.300000,125.000000,93.000000,81.000000,100.000000,193.000000,2.000000,4.000000,0.000000
50%,60.000000,27.100000,136.000000,102.000000,90.000000,124.000000,216.000000,3.000000,5.700000,1.000000
75%,68.000000,29.800000,148.000000,110.000000,99.000000,164.000000,239.250000,4.000000,7.300000,2.000000
max,90.000000,43.600000,195.000000,120.000000,120.000000,258.000000,328.000000,12.000000,10.000000,2.000000



Information about the test dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               3000 non-null   int64  
 1   gender            3000 non-null   object 
 2   bmi               3000 non-null   float64
 3   systolic_bp       3000 non-null   int64  
 4   diastolic_bp      3000 non-null   int64  
 5   heart_rate        3000 non-null   int64  
 6   blood_glucose     3000 non-null   int64  
 7   cholesterol       3000 non-null   int64  
 8   smoking_status    3000 non-null   object 
 9   exercise_level    3000 non-null   object 
 10  family_history    3000 non-null   object 
 11  medication_count  3000 non-null   int64  
 12  symptoms_score    3000 non-null   float64
 13  diagnosis         3000 non-null   object 
 14  diagnosis_code    3000 non-null   int64  
dtypes: float64(2), int64(8), object(5)
memory usage: 351

None


=== OUTLIER DETECTION ===
Outliers detected: 500 (5.00%)

Outliers by diagnosis class:
  Class 0: 194 outliers
  Class 1: 178 outliers
  Class 2: 128 outliers

Capping outliers to 1st/99th percentile...
  age: capped 179 values
  bmi: capped 191 values
  systolic_bp: capped 193 values
  diastolic_bp: capped 98 values
  heart_rate: capped 100 values
  blood_glucose: capped 97 values
  cholesterol: capped 192 values
  medication_count: capped 88 values
  symptoms_score: capped 97 values


## Check for missing values in the training dataset

In [4]:
print("Missing values in the training dataset:")
display(df_train.isnull().sum())

# Check class imbalance
print(f"\n=== CLASS BALANCE ANALYSIS ===")
class_percentages = df_train['diagnosis_code'].value_counts(normalize=True) * 100
print("Class distribution:")
for diagnosis, percentage in class_percentages.items():
    print(f"{diagnosis}: {percentage:.1f}%")



Missing values in the training dataset:


,0
age,0
gender,0
bmi,100
systolic_bp,0
diastolic_bp,0
heart_rate,0
blood_glucose,100
cholesterol,100
smoking_status,0
exercise_level,0



=== CLASS BALANCE ANALYSIS ===
Class distribution:
0: 61.1%
1: 29.3%
2: 9.6%


## Handle Missing Values with KNN Imputation

In [5]:
from sklearn.impute import KNNImputer
import numpy as np

# Select columns with missing values
cols_to_impute = ['bmi', 'blood_glucose', 'cholesterol', 'medication_count', 'symptoms_score']

# Initialize KNNImputer
# You can adjust the number of neighbors (n_neighbors)
imputer = KNNImputer(n_neighbors=5)

# Apply KNN imputation to the selected columns in the training dataset
df_train[cols_to_impute] = imputer.fit_transform(df_train[cols_to_impute])

# Apply the same imputer to the test dataset
df_test[cols_to_impute] = imputer.transform(df_test[cols_to_impute])


print("Missing values in the training dataset after KNN imputation:")
display(df_train[cols_to_impute].isnull().sum())

print("\nMissing values in the test dataset after KNN imputation:")
display(df_test[cols_to_impute].isnull().sum())

Missing values in the training dataset after KNN imputation:


,0
bmi,0
blood_glucose,0
cholesterol,0
medication_count,0
symptoms_score,0



Missing values in the test dataset after KNN imputation:


,0
bmi,0
blood_glucose,0
cholesterol,0
medication_count,0
symptoms_score,0


In [6]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Select categorical columns to one-hot encode
cols_to_encode = ['gender', 'smoking_status', 'exercise_level', 'family_history']

# Initialize OneHotEncoder
# handle_unknown='ignore' is used to handle categories that might be present in the test set but not in the training set.
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Apply one-hot encoding to the selected columns in the training dataset
encoded_train_data = encoder.fit_transform(df_train[cols_to_encode])
encoded_train_df = pd.DataFrame(encoded_train_data, columns=encoder.get_feature_names_out(cols_to_encode))

# Apply one-hot encoding to the selected columns in the test dataset
encoded_test_data = encoder.transform(df_test[cols_to_encode])
encoded_test_df = pd.DataFrame(encoded_test_data, columns=encoder.get_feature_names_out(cols_to_encode))

# Drop the original categorical columns
df_train = df_train.drop(cols_to_encode, axis=1)
df_test = df_test.drop(cols_to_encode, axis=1)

# Concatenate the one-hot encoded dataframes with the original dataframes
df_train = pd.concat([df_train, encoded_train_df], axis=1)
df_test = pd.concat([df_test, encoded_test_df], axis=1)

print("Training dataset after one-hot encoding:")
display(df_train.head())

print("\nTest dataset after one-hot encoding:")
display(df_test.head())

Training dataset after one-hot encoding:


,age,bmi,systolic_bp,diastolic_bp,heart_rate,blood_glucose,cholesterol,medication_count,symptoms_score,diagnosis,...,gender_F,gender_M,smoking_status_Current,smoking_status_Former,smoking_status_Never,exercise_level_High,exercise_level_Low,exercise_level_Moderate,family_history_No,family_history_Yes
0,37,33.8,100,74,108.0,206.0,145.0,1.0,2.90,Diabetes,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,51,21.6,141,95,88.0,72.0,204.0,6.0,5.80,Heart_Disease,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,64,29.3,122,101,63.0,108.0,228.0,7.0,6.60,Hypertension,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,40,33.7,103,81,75.0,206.0,195.0,3.0,8.60,Diabetes,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,68,20.4,110,78,88.0,156.0,183.0,4.0,6.14,Diabetes,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0



Test dataset after one-hot encoding:


,age,bmi,systolic_bp,diastolic_bp,heart_rate,blood_glucose,cholesterol,medication_count,symptoms_score,diagnosis,...,gender_F,gender_M,smoking_status_Current,smoking_status_Former,smoking_status_Never,exercise_level_High,exercise_level_Low,exercise_level_Moderate,family_history_No,family_history_Yes
0,43,24.4,132,99,89.00,97.0,248.0,4.0,5.6,Hypertension,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,53,23.3,148,103,89.00,140.0,238.0,2.0,5.9,Heart_Disease,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,58,25.0,100,76,58.99,217.0,242.0,3.0,5.1,Diabetes,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,43,31.7,120,99,73.00,89.0,205.0,2.0,10.0,Heart_Disease,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,50,28.1,138,103,105.00,197.0,140.0,4.0,4.1,Diabetes,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


## Scale Continuous Features

In [7]:
from sklearn.preprocessing import StandardScaler

# Identify continuous columns (excluding the target variable and one-hot encoded columns)
continuous_cols = ['age', 'bmi', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'blood_glucose', 'cholesterol', 'medication_count', 'symptoms_score']


# Initialize StandardScaler
scaler = StandardScaler()

# Apply StandardScaler to the continuous columns in the training dataset
df_train[continuous_cols] = scaler.fit_transform(df_train[continuous_cols])

# Apply the same scaler to the continuous columns in the test dataset
df_test[continuous_cols] = scaler.transform(df_test[continuous_cols])

print("Training dataset after scaling continuous features:")
display(df_train.head())

print("\nTest dataset after scaling continuous features:")
display(df_test.head())

Training dataset after scaling continuous features:


,age,bmi,systolic_bp,diastolic_bp,heart_rate,blood_glucose,cholesterol,medication_count,symptoms_score,diagnosis,...,gender_F,gender_M,smoking_status_Current,smoking_status_Former,smoking_status_Never,exercise_level_High,exercise_level_Low,exercise_level_Moderate,family_history_No,family_history_Yes
0,-1.647706,1.602146,-1.860183,-1.974542,1.436753,1.297484,-1.934563,-0.892835,-1.195550,Diabetes,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.499410,-1.386611,0.522856,-0.293331,-0.110030,-1.860204,-0.079351,2.098358,0.148088,Heart_Disease,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.566865,0.499735,-0.581479,0.187015,-2.043510,-1.011870,0.675311,2.696597,0.518747,Hypertension,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,-1.401643,1.577648,-1.685814,-1.414138,-1.115440,1.297484,-0.362350,0.303642,1.445394,Diabetes,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,0.894949,-1.680588,-1.278954,-1.654311,-0.110030,0.119242,-0.739681,0.901881,0.305618,Diabetes,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0



Test dataset after scaling continuous features:


,age,bmi,systolic_bp,diastolic_bp,heart_rate,blood_glucose,cholesterol,medication_count,symptoms_score,diagnosis,...,gender_F,gender_M,smoking_status_Current,smoking_status_Former,smoking_status_Never,exercise_level_High,exercise_level_Low,exercise_level_Moderate,family_history_No,family_history_Yes
0,-1.155580,-0.700667,-0.000250,0.026899,-0.032691,-1.271083,1.304196,0.901881,0.055424,Hypertension,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,-0.335368,-0.970145,0.929717,0.347130,-0.032691,-0.257795,0.989753,-0.294597,0.194421,Heart_Disease,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.074738,-0.553679,-1.860183,-1.814426,-2.353640,1.556698,1.115530,0.303642,-0.176238,Diabetes,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,-1.155580,1.087688,-0.697725,0.026899,-1.270118,-1.459602,-0.047907,-0.294597,2.094047,Heart_Disease,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,-0.581432,0.205759,0.348488,0.347130,1.204736,1.085401,-2.091784,0.901881,-0.639562,Diabetes,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


# Modeling
Create a deep neural network model using TensorFlow to train the provided dataset, which has already been loaded and preprocessed. The model should be trained on the training data and evaluated on the testing data.

## Prepare data

### Subtask:
Separate the features (X) and the target variable (y) for both the training and testing datasets.


**Reasoning**:
Separate the features (X) and the target variable (y) for both the training and testing datasets.



In [8]:
X_train = df_train.drop(['diagnosis', 'diagnosis_code'], axis=1)
y_train = df_train['diagnosis_code']

X_test = df_test.drop(['diagnosis', 'diagnosis_code'], axis=1)
y_test = df_test['diagnosis_code']

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (10000, 19)
Shape of y_train: (10000,)
Shape of X_test: (3000, 19)
Shape of y_test: (3000,)


In [9]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

print("Class weights to handle imbalance:")
for class_label, weight in class_weight_dict.items():
    print(f"Class {class_label}: weight = {weight:.3f}")

Class weights to handle imbalance:
Class 0: weight = 0.546
Class 1: weight = 1.137
Class 2: weight = 3.469


## Define model architecture

### IMPORTS AND SETUP

In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, LeakyReLU
from tensorflow.keras.optimizers import Adam, Nadam, SGD, RMSprop, Adamax
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, f1_score
import time


# Ensure data format (add this after your preprocessing)
X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_train = np.array(y_train).astype('int32')
y_test = np.array(y_test).astype('int32')


### MODEL 6: SIMPLE FAST ARCHITECTURE


In [ ]:
print("=== MODEL 6: SIMPLE FAST ARCHITECTURE ===")
start_time = time.time()

# Create simple model (fewer layers for faster training)
model_06_simple = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(3, activation='softmax')
])

# Compile
model_06_simple.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Quick training
early_stopping_06 = EarlyStopping(patience=5, restore_best_weights=True, verbose=1)

history_06 = model_06_simple.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping_06],
    verbose=1
)

# Evaluate
print("\n=== MODEL 6 RESULTS ===")
y_pred_06 = model_06_simple.predict(X_test)
y_pred_classes_06 = np.argmax(y_pred_06, axis=1)

# Metrics
accuracy_06 = np.mean(y_pred_classes_06 == y_test)
f1_06 = f1_score(y_test, y_pred_classes_06, average='macro')
class_2_recall_06 = np.mean(y_pred_classes_06[y_test == 2] == 2)

print(f"Accuracy: {accuracy_06:.4f}")
print(f"Macro F1: {f1_06:.4f}")
print(f"Class 2 Recall: {class_2_recall_06:.4f}")
print(f"Training time: {time.time() - start_time:.1f} seconds")

# Detailed report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_06, target_names=['Class 0', 'Class 1', 'Class 2']))

### MODEL 8: LEAKY RELU ACTIVATION

In [15]:
# ============================================
# MODEL 8: LEAKY RELU ACTIVATION
# ============================================

import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, f1_score
import time


print("=== MODEL 8: LEAKY RELU ACTIVATION ===")
start_time = time.time()

# Create model with Leaky ReLU
model_08_leaky = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Compile
model_08_leaky.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Quick training
early_stopping_08 = EarlyStopping(patience=5, restore_best_weights=True, verbose=1)

history_08 = model_08_leaky.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping_08],
    verbose=1
)

# Evaluate
print("\n=== MODEL 8 RESULTS ===")
y_pred_08 = model_08_leaky.predict(X_test)
y_pred_classes_08 = np.argmax(y_pred_08, axis=1)

# Metrics
accuracy_08 = np.mean(y_pred_classes_08 == y_test)
f1_08 = f1_score(y_test, y_pred_classes_08, average='macro')
class_2_recall_08 = np.mean(y_pred_classes_08[y_test == 2] == 2)

print(f"Accuracy: {accuracy_08:.4f}")
print(f"Macro F1: {f1_08:.4f}")
print(f"Class 2 Recall: {class_2_recall_08:.4f}")
print(f"Training time: {time.time() - start_time:.1f} seconds")

# Detailed report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_08, target_names=['Class 0', 'Class 1', 'Class 2']))

=== MODEL 8: LEAKY RELU ACTIVATION ===


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 28s 177ms/step - accuracy: 0.6256 - loss: 0.8604 - val_accuracy: 0.8553 - val_loss: 0.4510
Epoch 2/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 39s 168ms/step - accuracy: 0.8557 - loss: 0.4182 - val_accuracy: 0.8687 - val_loss: 0.3472
Epoch 3/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 46s 201ms/step - accuracy: 0.8684 - loss: 0.4070 - val_accuracy: 0.8680 - val_loss: 0.3332
Epoch 4/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 26s 168ms/step - accuracy: 0.8713 - loss: 0.3828 - val_accuracy: 0.8677 - val_loss: 0.3290
Epoch 5/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 23s 147ms/step - accuracy: 0.8827 - loss: 0.3813 - val_accuracy: 0.8673 - val_loss: 0.3318
Epoch 6/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 48s 191ms/step - accuracy: 0.8802 - loss: 0.3618 - val_accuracy: 0.8700 - val_loss: 0.3281
Epoch 7/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 167ms/step - accuracy: 0.8854 - loss: 0.3553 - val_accuracy: 0.8717 - val_loss: 0.3312
Epoch 8/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 170ms/step - accuracy: 0.8814 - loss: 0

### MODEL 9: HEAVY REGULARIZATION

In [14]:
# ============================================
# MODEL 9: HEAVY REGULARIZATION
# ============================================

import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report, f1_score
import time

print("=== MODEL 9: HEAVY REGULARIZATION ===")
start_time = time.time()

# Create heavily regularized model
model_09_regularized = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.4),  # Higher dropout
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),  # Even higher dropout
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

# Compile
model_09_regularized.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Quick training
early_stopping_09 = EarlyStopping(patience=5, restore_best_weights=True, verbose=1)

history_09 = model_09_regularized.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping_09],
    verbose=1
)

# Evaluate
print("\n=== MODEL 9 RESULTS ===")
y_pred_09 = model_09_regularized.predict(X_test)
y_pred_classes_09 = np.argmax(y_pred_09, axis=1)

# Metrics
accuracy_09 = np.mean(y_pred_classes_09 == y_test)
f1_09 = f1_score(y_test, y_pred_classes_09, average='macro')
class_2_recall_09 = np.mean(y_pred_classes_09[y_test == 2] == 2)

print(f"Accuracy: {accuracy_09:.4f}")
print(f"Macro F1: {f1_09:.4f}")
print(f"Class 2 Recall: {class_2_recall_09:.4f}")
print(f"Training time: {time.time() - start_time:.1f} seconds")

# Detailed report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_09, target_names=['Class 0', 'Class 1', 'Class 2']))

=== MODEL 9: HEAVY REGULARIZATION ===
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 26s 162ms/step - accuracy: 0.4964 - loss: 4.3109 - val_accuracy: 0.8603 - val_loss: 3.1401
Epoch 2/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.7974 - loss: 2.9514 - val_accuracy: 0.8600 - val_loss: 2.2665
Epoch 3/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.8475 - loss: 2.1773 - val_accuracy: 0.8610 - val_loss: 1.6684
Epoch 4/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.8587 - loss: 1.6315 - val_accuracy: 0.8693 - val_loss: 1.2570
Epoch 5/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 158ms/step - accuracy: 0.8707 - loss: 1.2448 - val_accuracy: 0.8680 - val_loss: 0.9723
Epoch 6/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 25s 157ms/step - accuracy: 0.8790 - loss: 0.9776 - val_accuracy: 0.8680 - val_loss: 0.7848
Epoch 7/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.8724 - loss: 0.8097 - val_accuracy: 0.8693 - val_loss: 0.6633
Epoch 8/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 40s 155ms/step - accuracy: 0.8827 - loss: 0.6920 - val